<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parse-the-HTML" data-toc-modified-id="Parse-the-HTML-1">Parse the HTML</a></span></li><li><span><a href="#Get-all-the-team-names" data-toc-modified-id="Get-all-the-team-names-2">Get all the team names</a></span></li><li><span><a href="#Get-the-headers" data-toc-modified-id="Get-the-headers-3">Get the headers</a></span></li><li><span><a href="#Get-all-the-data" data-toc-modified-id="Get-all-the-data-4">Get all the data</a></span></li><li><span><a href="#Wrap-the-data-all-in-one-DataFrame" data-toc-modified-id="Wrap-the-data-all-in-one-DataFrame-5">Wrap the data all in one DataFrame</a></span></li></ul></div>

## Parse the HTML

In [46]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
url = 'https://fbref.com/en/comps/9/2020-2021/stats/2020-2021-Premier-League-Stats'

In [11]:
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

In [31]:
table = soup.find_all('table')[0]

In [32]:
print(table)

<table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_squads_standard_for"> <caption>Squad Standard Stats <span style="color: #666; font-size:smaller">2020-2021 Premier League</span> Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr class="over_header"> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_playing">Playing Time</th> <th aria-label="" class="over_header center group_start" colspan="8" data-stat="header_performance">Performance</th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Expected</th> <th aria-label="" class="over_header center group_start" colspan="2" data-stat="header_progression">Progressi

## Get all the team names

In [38]:
team_names_list = table.find('tbody').find_all('th')

In [41]:
team_names = [name.text for name in team_names_list]
print(team_names)

['Arsenal', 'Aston Villa', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds United', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Sheffield Utd', 'Southampton', 'Tottenham', 'West Brom', 'West Ham', 'Wolves']


## Get the headers

In [84]:
html = table.find('tbody').find_all('tr')[0]
html_str = str(html)

In [85]:
header_list = re.findall(r"data-stat=\"(.*?)\"",html_str)
print(header_list)

['team', 'players_used', 'avg_age', 'possession', 'games', 'games_starts', 'minutes', 'minutes_90s', 'goals', 'assists', 'goals_assists', 'goals_pens', 'pens_made', 'pens_att', 'cards_yellow', 'cards_red', 'xg', 'npxg', 'xg_assist', 'npxg_xg_assist', 'progressive_carries', 'progressive_passes', 'goals_per90', 'assists_per90', 'goals_assists_per90', 'goals_pens_per90', 'goals_assists_pens_per90', 'xg_per90', 'xg_assist_per90', 'xg_xg_assist_per90', 'npxg_per90', 'npxg_xg_assist_per90']


## Get all the data

In [81]:
# print([float(item.text.replace(',', '')) for item in html.find_all('td')]) # first row as an example

[29.0, 25.9, 53.5, 38.0, 418.0, 3420.0, 38.0, 53.0, 38.0, 91.0, 47.0, 6.0, 6.0, 49.0, 5.0, 51.8, 47.0, 35.1, 82.1, 758.0, 1760.0, 1.39, 1.0, 2.39, 1.24, 2.24, 1.36, 0.92, 2.29, 1.24, 2.16]


In [93]:
data = []
i = 0
for row in table.find('tbody').find_all('tr'):
    digits = [float(item.text.replace(',', '')) for item in row.find_all('td')]
    full_list = [team_names[i]]+digits
    data.append(full_list)
    i+=1

In [94]:
print(data)

[['Arsenal', 29.0, 25.9, 53.5, 38.0, 418.0, 3420.0, 38.0, 53.0, 38.0, 91.0, 47.0, 6.0, 6.0, 49.0, 5.0, 51.8, 47.0, 35.1, 82.1, 758.0, 1760.0, 1.39, 1.0, 2.39, 1.24, 2.24, 1.36, 0.92, 2.29, 1.24, 2.16], ['Aston Villa', 24.0, 25.2, 48.5, 38.0, 418.0, 3420.0, 38.0, 52.0, 38.0, 90.0, 47.0, 5.0, 6.0, 71.0, 4.0, 52.5, 47.9, 37.4, 85.3, 629.0, 1420.0, 1.37, 1.0, 2.37, 1.24, 2.24, 1.38, 0.98, 2.37, 1.26, 2.25], ['Brighton', 27.0, 25.8, 51.1, 38.0, 418.0, 3420.0, 38.0, 39.0, 24.0, 63.0, 33.0, 6.0, 9.0, 49.0, 6.0, 50.9, 43.8, 33.2, 77.0, 685.0, 1516.0, 1.03, 0.63, 1.66, 0.87, 1.5, 1.34, 0.87, 2.21, 1.15, 2.03], ['Burnley', 25.0, 28.3, 42.1, 38.0, 418.0, 3420.0, 38.0, 32.0, 20.0, 52.0, 29.0, 3.0, 3.0, 48.0, 0.0, 39.3, 36.9, 27.7, 64.6, 389.0, 1012.0, 0.84, 0.53, 1.37, 0.76, 1.29, 1.03, 0.73, 1.76, 0.97, 1.7], ['Chelsea', 27.0, 26.0, 60.9, 38.0, 418.0, 3420.0, 38.0, 56.0, 38.0, 94.0, 48.0, 8.0, 10.0, 51.0, 3.0, 62.4, 54.5, 41.5, 96.1, 895.0, 1798.0, 1.47, 1.0, 2.47, 1.26, 2.26, 1.64, 1.09, 2.74, 1

## Wrap the data all in one DataFrame

In [97]:
df = pd.DataFrame(data=data,columns=header_list)
df.head(10)

,team,players_used,avg_age,possession,games,games_starts,minutes,minutes_90s,goals,assists,...,goals_per90,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90
0,Arsenal,29.0,25.9,53.5,38.0,418.0,3420.0,38.0,53.0,38.0,...,1.39,1.00,2.39,1.24,2.24,1.36,0.92,2.29,1.24,2.16
1,Aston Villa,24.0,25.2,48.5,38.0,418.0,3420.0,38.0,52.0,38.0,...,1.37,1.00,2.37,1.24,2.24,1.38,0.98,2.37,1.26,2.25
2,Brighton,27.0,25.8,51.1,38.0,418.0,3420.0,38.0,39.0,24.0,...,1.03,0.63,1.66,0.87,1.50,1.34,0.87,2.21,1.15,2.03
3,Burnley,25.0,28.3,42.1,38.0,418.0,3420.0,38.0,32.0,20.0,...,0.84,0.53,1.37,0.76,1.29,1.03,0.73,1.76,0.97,1.70
4,Chelsea,27.0,26.0,60.9,38.0,418.0,3420.0,38.0,56.0,38.0,...,1.47,1.00,2.47,1.26,2.26,1.64,1.09,2.74,1.44,2.53
5,Crystal Palace,24.0,29.1,40.6,38.0,418.0,3420.0,38.0,39.0,29.0,...,1.03,0.76,1.79,0.95,1.71,0.90,0.62,1.52,0.82,1.44
6,Everton,29.0,26.3,46.4,38.0,418.0,3420.0,38.0,45.0,32.0,...,1.18,0.84,2.03,1.08,1.92,1.20,0.81,2.01,1.10,1.91
7,Fulham,28.0,25.2,49.7,38.0,418.0,3420.0,38.0,26.0,18.0,...,0.68,0.47,1.16,0.61,1.08,1.07,0.69,1.76,0.95,1.64
8,Leeds United,23.0,26.1,57.3,38.0,418.0,3420.0,38.0,60.0,45.0,...,1.58,1.18,2.76,1.47,2.66,1.46,1.10,2.57,1.38,2.48
9,Leicester City,27.0,26.5,54.3,38.0,418.0,3420.0,38.0,64.0,45.0,...,1.68,1.18,2.87,1.42,2.61,1.47,0.89,2.36,1.22,2.11


In [99]:
# Output the dataset
df.to_excel(r'E:\Data\Football_Viz\Dataset\20-21PremierLeague SquadStdStats.xlsx',index=False)